In [13]:
import numpy as np
import pandas as pd
import pyproj
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import shape, Polygon, Point
from shapely.ops import transform

In [2]:
urldistr = 'https://tsb-opendata.s3.eu-central-1.amazonaws.com/bezirksgrenzen/bezirksgrenzen.geojson'
distr_gdf = gpd.read_file(urldistr)
distr_gdf = distr_gdf[['Gemeinde_name', 'Land_name','geometry']]
distr_gdf = distr_gdf.rename(columns={'Gemeinde_name':'District Name', 'Land_name':'State Name'})
distr_gdf = distr_gdf.set_crs(epsg=4326, allow_override=True)
distr_gdf

,District Name,State Name,geometry
0,Reinickendorf,Berlin,"MULTIPOLYGON (((13.32074 52.62660, 13.32045 52..."
1,Charlottenburg-Wilmersdorf,Berlin,"MULTIPOLYGON (((13.32111 52.52446, 13.32103 52..."
2,Treptow-Köpenick,Berlin,"MULTIPOLYGON (((13.57925 52.39083, 13.57958 52..."
3,Pankow,Berlin,"MULTIPOLYGON (((13.50481 52.61960, 13.50467 52..."
4,Neukölln,Berlin,"MULTIPOLYGON (((13.45832 52.48569, 13.45823 52..."
5,Lichtenberg,Berlin,"MULTIPOLYGON (((13.52316 52.47424, 13.52367 52..."
6,Marzahn-Hellersdorf,Berlin,"MULTIPOLYGON (((13.61317 52.47213, 13.61329 52..."
7,Spandau,Berlin,"MULTIPOLYGON (((13.23119 52.56124, 13.23092 52..."
8,Steglitz-Zehlendorf,Berlin,"MULTIPOLYGON (((13.31952 52.46699, 13.31910 52..."
9,Mitte,Berlin,"MULTIPOLYGON (((13.37721 52.56091, 13.37715 52..."


In [3]:
distr_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [3]:
berlin_polyg = distr_gdf[['State Name','geometry']].dissolve(by='State Name')

In [4]:
berlin_polyg

,geometry
State Name,
Berlin,"MULTIPOLYGON (((13.56770 52.57451, 13.56772 52..."


In [5]:
berlin_polyg_utm = berlin_polyg.to_crs(epsg=32633)
berlin_polyg_utm

,geometry
State Name,
Berlin,"MULTIPOLYGON (((402938.488 5825903.409, 402939..."


In [15]:
berlin_shape = shape(berlin_polyg['geometry'])
berlin_polyg_utm1 = transform(trans2xy, berlin_shape)

ValueError: Unknown geometry type: featurecollection

In [7]:
berlin_polyg_utm.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
address = 'Berlin Germany'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
berlin = [location.longitude, location.latitude]

In [9]:
proj2xy = pyproj.Proj("epsg:32633", preserve_units=False)
x, y = proj2xy(berlin[0], berlin[1])
lon, lat = proj2xy(x,y, inverse=True)
print('Berlin coordinate, Lon= {}; Lat= {}'.format(berlin[0], berlin[1]))
print('Berlin transform to UTM, X= {}; Y= {}'.format(x,y))
print('Berlin reverse transform, Lon= {}; Lat= {}'.format(lon, lat))

Berlin coordinate, Lon= 13.3888599; Lat= 52.5170365
Berlin transform to UTM, X= 390676.87103088724; Y= 5819766.874314442
Berlin reverse transform, Lon= 13.3888599; Lat= 52.51703649999999


In [10]:
transform2xy = pyproj.Transformer.from_crs("+proj=latlong +datum=WGS84 +ellps=WGS84" , "+proj=utm +zone=33 +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0")
transform2ll = pyproj.Transformer.from_crs("+proj=utm +zone=33 +datum=WGS84 +ellps=WGS84" , "+proj=latlong +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0")
x, y = transform2xy.transform(berlin[0], berlin[1])
lon, lat = transform2ll.transform(x,y)
print('Berlin center coordinate, Lon= {}; Lat= {}'.format(berlin[0], berlin[1]))
print('Berlin center transform to UTM, X= {}; Y= {}'.format(x,y))
print('Berlin center reverse transform, Lon= {}; Lat= {}'.format(lon, lat))

Berlin center coordinate, Lon= 13.3888599; Lat= 52.5170365
Berlin center transform to UTM, X= 390676.87103088724; Y= 5819766.874314442
Berlin center reverse transform, Lon= 13.3888599; Lat= 52.51703649999999


In [7]:
trans2xy = pyproj.Transformer.from_crs("epsg:4326","epsg:32633", always_xy=True)
trans2ll = pyproj.Transformer.from_crs("epsg:32633","epsg:4326", always_xy=True)
x, y = trans2xy.transform(berlin[0], berlin[1])
lon, lat = trans2ll.transform(x,y)
print('Berlin center coordinate, Lon= {}; Lat= {}'.format(berlin[0], berlin[1]))
print('Berlin center transform to UTM, X= {}; Y= {}'.format(x,y))
print('Berlin center reverse transform, Lon= {}; Lat= {}'.format(lon, lat))

Berlin center coordinate, Lon= 13.3888599; Lat= 52.5170365
Berlin center transform to UTM, X= 390676.87103088724; Y= 5819766.874314442
Berlin center reverse transform, Lon= 13.3888599; Lat= 52.51703649999999


In [13]:
utmbounds = berlin_polyg_utm.total_bounds
print (utmbounds[2], utmbounds[3])

415786.55400086974 5837259.270123978


In [23]:
llbounds = berlin_polyg.total_bounds
maxx, maxy = trans2xy.transform(llbounds[2], llbounds[3])
print (maxx, maxy)

416240.67868526874 5836894.550149376


In [28]:
maxx2, maxy2 = transform2xy.transform(llbounds[2],llbounds[3])
print (maxx2, maxy2)

416240.67868526874 5836894.550149376


In [24]:
minlon = berlin_polyg.total_bounds[0]
maxlon = berlin_polyg.total_bounds[2]
minlat = berlin_polyg.total_bounds[1]
maxlat = berlin_polyg.total_bounds[3]
print("Min Lon = {}\nMax Lon = {}\nMin Lat = {}\nMax Lat = {}".format(minlon ,maxlon ,minlat, maxlat))
berlin_x_range = maxlon - minlon
berlin_y_range = maxlat - minlat
print("The length of Berlin's X range is {} decimal degrees and Y range is {} \ndecimal degrees.".format(
    berlin_x_range, berlin_y_range))
berlin_x_radius = berlin_x_range/2
berlin_y_radius = berlin_y_range/2
print("The length of Berlin's X radius is {} decimal degrees and Y radius is {} decimal degrees.".format(
    berlin_x_radius, berlin_y_radius))

Min Lon = 13.088347614730992
Max Lon = 13.761159145035254
Min Lat = 52.338245549997296
Max Lat = 52.67550765972353
The length of Berlin's X range is 0.6728115303042621 decimal degrees and Y range is 0.3372621097262325 
decimal degrees.
The length of Berlin's X radius is 0.33640576515213105 decimal degrees and Y radius is 0.16863105486311625 decimal degrees.


In [25]:
minx = berlin_polyg_utm.bounds.iloc[0,0]
maxx = berlin_polyg_utm.bounds.iloc[0,2]
miny = berlin_polyg_utm.bounds.iloc[0,1]
maxy = berlin_polyg_utm.bounds.iloc[0,3]
print("Min X = {}\nMax X = {}\nMin Y = {}\nMax Y = {}".format(minx ,maxx ,miny, maxy))
berlin_x_range = maxx - minx
berlin_y_range = maxy - miny
print("The length of Berlin's X range is {} meters and Y range is {} meters.".format(
    berlin_x_range, berlin_y_range))
berlin_x_radius = berlin_x_range/2
berlin_y_radius = berlin_y_range/2
print("The length of Berlin's X radius is {} meters and Y radius is {} meters.".format(
    berlin_x_radius, berlin_y_radius))

Min X = 370000.82600134326
Max X = 415786.55400086974
Min Y = 5799520.662123909
Max Y = 5837259.270123978
The length of Berlin's X range is 45785.72799952648 meters and Y range is 37738.608000068925 meters.
The length of Berlin's X radius is 22892.86399976324 meters and Y radius is 18869.304000034463 meters.


In [26]:
minx1, miny1 = trans2xy.transform(minlon, minlat) 
maxx1, maxy1 = trans2xy.transform(maxlon, maxlat)
print("Min X = {}\nMax X = {}\nMin Y = {}\nMax Y = {}".format(minx1 ,maxx1 ,miny1, maxy1))
berlin_x_range1 = maxx1 - minx1
berlin_y_range1 = maxy1 - miny1
print("The length of Berlin's X range is {} meters and Y range is {} meters.".format(
    berlin_x_range1, berlin_y_range1))
berlin_x_radius1 = berlin_x_range1/2
berlin_y_radius1 = berlin_y_range1/2
print("The length of Berlin's X radius is {} meters and Y radius is {} meters.".format(
    berlin_x_radius1, berlin_y_radius1))

Min X = 369761.53985129925
Max X = 416240.67868526874
Min Y = 5800380.171821327
Max Y = 5836894.550149376
The length of Berlin's X range is 46479.13883396948 meters and Y range is 36514.378328048624 meters.
The length of Berlin's X radius is 23239.56941698474 meters and Y radius is 18257.189164024312 meters.


In [29]:
minx2, miny2 = transform2xy.transform(minlon, minlat) 
maxx2, maxy2 = transform2xy.transform(maxlon, maxlat)
print("Min X = {}\nMax X = {}\nMin Y = {}\nMax Y = {}".format(minx2 ,maxx2 ,miny2, maxy2))
berlin_x_range2 = maxx2 - minx2
berlin_y_range2 = maxy2 - miny2
print("The length of Berlin's X range is {} meters and Y range is {} meters.".format(
    berlin_x_range2, berlin_y_range2))
berlin_x_radius2 = berlin_x_range2/2
berlin_y_radius2 = berlin_y_range2/2
print("The length of Berlin's X radius is {} meters and Y radius is {} meters.".format(
    berlin_x_radius2, berlin_y_radius2))

Min X = 369761.53985129925
Max X = 416240.67868526874
Min Y = 5800380.171821327
Max Y = 5836894.550149376
The length of Berlin's X range is 46479.13883396948 meters and Y range is 36514.378328048624 meters.
The length of Berlin's X radius is 23239.56941698474 meters and Y radius is 18257.189164024312 meters.
